In [35]:
import os
from os.path import join, splitext, basename, dirname, abspath
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn as sk
%matplotlib inline

In [36]:
from keras.models import load_model, Sequential, MODEL_DIR, NOTHING_DIR, REBAL_DIR

In [37]:
import trading as trd
from trading import StockPair, State

In [47]:
# relaod module after modifying it
import importlib
importlib.reload(trd)
importlib.reload(trd.stock_history)
importlib.reload(trd.portfolio)
importlib.reload(trd.benchmarks)
importlib.reload(trd.rl)
importlib.reload(trd.nn)
importlib.reload(trd)

<module 'trading' from 'C:\\Users\\hamza\\Documents\\school\\cs_229\\cs229_final_project\\src\\trading\\__init__.py'>

In [39]:
MODEL_DIR_NAME = '6days_pen_reward_l=0.5_100h_relu_15eps_1000000init'
MODEL_DIR = join(MODEL_DATA_LOC, MODEL_DIR_NAME)

## Portfolio Setup

In [40]:
# assume same across all models
cash = 1e6
starting_weights = (0.5, 0.5)
trans_cost = 0.001

In [41]:
# rebalance frequency
rebalance_period = 30

In [42]:
# number days input, from the file name
d = 7
reward = trd.sharpe_ratio_reward

## Load Test Set (per the model)

In [ ]:
# load the test set stock pairs from the saved file
stock_pairs = []

with open(join(MODEL_DIR, 'test.csv'), 'r') as f:
    for line in f:
        (L, H) = line.strip().split(',')
        hist_lo = trd.read_stock_history(join(trd.LO_BETA_DIR, L + '.csv'))
        hist_hi = trd.read_stock_history(join(trd.HI_BETA_DIR, H + '.csv'))

        stock_pairs.append(StockPair(L, H, hist_lo, hist_hi))

In [ ]:
test_states = [State(p, cash=cash, target_weights=starting_weights, trans_cost=trans_cost, d=d) for p in stock_pairs]
available_test_states = test_states[:]

## Load the Model

In [53]:
model = load_model(join(MODEL_DIR, 'model.h5'))

Exception: Invalid initialization: my_init

## Evaluate Model

In [ ]:
# run the model on each test portfolio
while True:
    if available_test_states == []:
        break
    
    states = np.array([st.state for st in available_test_states])
 
    qvalues = model.predict(states)
    chosen_actions = np.argmax(qvalues, axis=1)
    
    for (st, a) in zip(available_test_states, trd.actions[chosen_actions]):
        # execute the action
        st.execute_trade(a)

        try:
            st.step()
        except StopIteration:
            available_test_states.remove(st)

In [ ]:
# save off the test runs
for st in test_states:
    file_name = '{:s}_{:s}.csv'.format(st.lo.symbol, st.hi.symbol)
    st.portfolio.to_csv(join(MODEL_DIR, file_name))

## Run Benchmarks

In [ ]:
do_nothing_portfolios = []
rebalance_portfolios = []

for st in test_states:
    nothing = trd.do_nothing_benchmark(lo, hi, 
                                       initial_value=cash
                                       trans_cost=trans_cost)
    rebal = trd.rebalance_benchmark(lo, hi, rebalance_period=rebalance_period, 
                                    initial_value=cash, 
                                    trans_cost=trans_cost)
    
    do_nothing_portfolios.append(nothing)
    rebalance_portfolios.append(rebal)
    
    nothing.to_csv(join(MODEL_DIR, NOTHIN_DIR, '{:s}_{:s}.csv'.format(st.lo, st.hi)))
    rebal.to_csv(join(MODEL_DIR, REBAL_DIR, '{:s}_{:s}.csv'.format(st.lo, st.hi)))


In [ ]:
assert len(do_nothing_portfolios) = len(test_states)
assert len(rebalance_portfolios) = len(test_states)

In [ ]:
def cust_plt(ax, data, label='', **kwargs):
    ax.plot_date(data.index, data, ls='solid', marker='', label=label, **kwargs)

In [ ]:
# number of test examples
N = len(test_states)

matplotlib.rc('font', size=16)

for i, st in enumerate(test_states):
    f = plt.figure(figsize=(16, 10))

    lo_name = st.lo.symbol
    hi_name = st.hi.symbol
    
    do_nothing = do_nothing_portfolios[i]
    rebal = rebalance_portfolios[i]

    # plot portfolio over time
    ax1 = plt.subplot(1, 2, 1)
    lbl_str = '{:s} ({:.5f})'
    cust_plt(ax1, do_nothing.total, lbl_str.format('do nothing', reward(do_nothing)))
    cust_plt(ax1, rebal.total, lbl_str.format('rebalance {:d}'.format(rebalance_period), reward(rebal)))
    cust_plt(ax1, st.portfolio.total, lbl_str.format('Q Learning', reward(st.portfolio)))
    ax1.axhline(y=cash, color='black')
    ax1.legend(bbox_to_anchor=(0, 1), loc='upper left', ncol=1)
    ax1.set_ylim(bottom=0)
    plt.xticks(rotation=40)
    
    # histogram of returns
    port_returns = st.portfolio.ix[1:, 'total'] - st.portfolio.ix[:-1, 'total']
    rebal_returns = rebal.ix[1:, 'total'] - rebal.ix[:-1, 'total']
    nothing_returns = do_nothing.ix[1:, 'total'] - do_nothing.ix[:-1, 'total']
    
    assert False, 'implement me'
    
    ax2 = plt.subplot(1, 2, 2)
    f.set_title('{:s} vs {:s}'.format(lo_name, hi_name))
    f.savefig(join(MODEL_DIR, '{:s}_{:s}_cmp.png'.format(lo_name, hi_name)))